# Loop 45 Analysis: Strategic Assessment

## Key Questions:
1. What is the current CV-LB relationship?
2. Why is the intercept (0.0533) > target (0.0347)?
3. What approaches haven't been tried that could reduce the intercept?
4. Should we submit exp_046 (similarity features) or try something else?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
    {'exp': 'exp_041', 'cv': 0.0090, 'lb': 0.0932},
    {'exp': 'exp_042', 'cv': 0.0145, 'lb': 0.1147},
]

df = pd.DataFrame(submissions)
print('Submission History:')
print(df.to_string(index=False))

In [ ]:
# Fit CV-LB relationship
from scipy import stats

cv = df['cv'].values
lb = df['lb'].values

slope, intercept, r_value, p_value, std_err = stats.linregress(cv, lb)

print(f'\n=== CV-LB Relationship ===')
print(f'LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'\nIntercept: {intercept:.4f}')
print(f'Target: 0.0347')
print(f'Gap: {intercept - 0.0347:.4f} ({(intercept - 0.0347)/0.0347*100:.1f}% above target)')

# What CV would be needed to reach target?
cv_needed = (0.0347 - intercept) / slope
print(f'\nCV needed to reach target: {cv_needed:.6f}')
if cv_needed < 0:
    print('IMPOSSIBLE: Even with CV=0, LB would be above target!')

In [ ]:
# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(cv, lb, s=100, c='blue', alpha=0.7, label='Submissions')

# Fit line
cv_range = np.linspace(0, max(cv)*1.1, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f}')

# Target line
plt.axhline(y=0.0347, color='green', linestyle=':', linewidth=2, label='Target (0.0347)')

# Intercept line
plt.axhline(y=intercept, color='orange', linestyle='--', alpha=0.5, label=f'Intercept ({intercept:.4f})')

plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV-LB Relationship: The Intercept Problem')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print('\nKEY INSIGHT: The intercept (0.0533) > target (0.0347)')
print('This means we CANNOT reach the target by improving CV alone.')
print('We need to find an approach with a LOWER INTERCEPT.')

In [ ]:
# Analyze recent experiments
recent_experiments = [
    {'exp': 'exp_032', 'cv': 0.008194, 'description': 'GP(0.15)+MLP(0.55)+LGBM(0.3) - Best CV, NOT submitted'},
    {'exp': 'exp_041', 'cv': 0.008199, 'description': 'Same as exp_032, submitted'},
    {'exp': 'exp_042', 'cv': 0.009237, 'description': 'GroupKFold(5) CV - Different CV procedure'},
    {'exp': 'exp_043', 'cv': 0.009002, 'description': 'Aggressive regularization'},
    {'exp': 'exp_044', 'cv': 0.014503, 'description': 'Pure GP'},
    {'exp': 'exp_045', 'cv': 0.010001, 'description': 'Stacking'},
    {'exp': 'exp_046', 'cv': 0.008717, 'description': 'Similarity Features'},
]

print('\n=== Recent Experiments ===')
for exp in recent_experiments:
    predicted_lb = slope * exp['cv'] + intercept
    print(f"{exp['exp']}: CV={exp['cv']:.6f}, Predicted LB={predicted_lb:.4f} - {exp['description']}")

In [ ]:
# What approaches haven't been tried?
print('\n=== APPROACHES NOT YET TRIED ===')
print()
print('1. CATBOOST with categorical features')
print('   - Different boosting algorithm that handles categoricals natively')
print('   - Might have different extrapolation behavior')
print()
print('2. DIFFERENT FEATURE REPRESENTATIONS')
print('   - Raw RDKit descriptors (200+ features)')
print('   - Morgan fingerprints (different from DRFP)')
print('   - Pre-trained molecular embeddings')
print()
print('3. DOMAIN ADAPTATION')
print('   - Adversarial training to reduce distribution shift')
print('   - Sample weighting based on similarity to test distribution')
print()
print('4. ENSEMBLE WITH DIFFERENT FEATURE SETS')
print('   - Train separate models on different feature sets')
print('   - Combine with learned weights')
print()
print('5. INTERACTION FEATURES')
print('   - Temperature * Residence Time (Reaction Energy)')
print('   - SolventB% * Temperature')
print('   - These were used in the "mr0106/catechol" kernel')

In [ ]:
# Analyze the similarity features experiment
print('\n=== SIMILARITY FEATURES ANALYSIS ===')
print()
print('exp_046 (Similarity Features):')
print(f'  CV: 0.008717 (6.38% worse than best CV 0.008194)')
print(f'  Predicted LB: {slope * 0.008717 + intercept:.4f}')
print()
print('The similarity features HURT CV performance.')
print('This suggests they are adding noise rather than signal.')
print()
print('RECOMMENDATION: DO NOT submit exp_046.')
print('Instead, try approaches that could fundamentally change the CV-LB relationship.')

In [ ]:
# What would it take to reach the target?
print('\n=== PATH TO TARGET ===')
print()
print(f'Target: 0.0347')
print(f'Best LB: 0.0877')
print(f'Gap: {0.0877 - 0.0347:.4f} ({(0.0877 - 0.0347)/0.0347*100:.1f}% above target)')
print()
print('Current CV-LB relationship: LB = 4.21*CV + 0.0533')
print()
print('To reach target with current relationship:')
print(f'  Need CV = (0.0347 - 0.0533) / 4.21 = {(0.0347 - 0.0533) / 4.21:.6f}')
print('  IMPOSSIBLE: CV cannot be negative!')
print()
print('To reach target, we need to CHANGE THE RELATIONSHIP:')
print()
print('Option 1: Reduce intercept to ~0.03')
print('  If intercept = 0.03, then with CV = 0.001, LB = 0.034')
print('  This requires finding features/models that generalize better')
print()
print('Option 2: Reduce slope to ~1.0')
print('  If slope = 1.0 and intercept = 0.03, then with CV = 0.005, LB = 0.035')
print('  This requires reducing the CV-LB gap')
print()
print('Option 3: Find a completely different approach')
print('  The target (0.0347) exists, so someone achieved it')
print('  What are they doing differently?')

In [ ]:
# Strategic recommendations
print('\n=== STRATEGIC RECOMMENDATIONS ===')
print()
print('With 3 submissions remaining, we need to be strategic.')
print()
print('PRIORITY 1: Try CatBoost with categorical features')
print('  - Different algorithm might have different CV-LB relationship')
print('  - Use solvent name as categorical feature')
print('  - Add interaction features (Temp*Time, SolventB%*Temp)')
print()
print('PRIORITY 2: Try XGBoost + RF + LightGBM ensemble (from mixall kernel)')
print('  - The mixall kernel claims good CV/LB')
print('  - Uses GroupKFold(5) which might be more realistic')
print('  - Ensemble of 4 models with learned weights')
print()
print('PRIORITY 3: Submit best CV model (exp_032) if nothing else works')
print('  - CV 0.008194 is the best we have')
print('  - Predicted LB: 0.0880 (similar to best LB 0.0877)')
print('  - At least confirms the CV-LB relationship')
print()
print('DO NOT SUBMIT:')
print('  - exp_046 (Similarity Features): CV is worse, no evidence of different relationship')
print('  - exp_045 (Stacking): CV is 22% worse')
print('  - exp_044 (Pure GP): CV is 77% worse')